Faça o upload de um arquivo .zip com os seus dados e use o comando abaixo após a finalização do upload para a extração do conteúdo do arquivo no colab.

Implemente as funções a seguir.

In [16]:
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn
import os
import pandas as pd
from torchvision.io import read_image
from torchvision import transforms


In [17]:

class CarSimDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0].strip())
        image = read_image(img_path) 
        label = self.img_labels.iloc[idx, 1]
        
        if self.transform:
            image = self.transform(image)  
        
        if self.target_transform:
            label = self.target_transform(label)
        
        return image, label


Crie as variáveis a seguir a partir da classe implementada acima.

In [18]:
# Ajuste nas transformações para assegurar que o tensor seja convertido para float, normalizado e com aumento de dados aplicado
transform = transforms.Compose([
    transforms.Resize((28, 28)),  
    transforms.RandomRotation(degrees=5),  
    transforms.ColorJitter(brightness=0.2), 
    transforms.RandomHorizontalFlip(),  
    transforms.ConvertImageDtype(torch.float), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

# Transformação dos rótulos (normalizando os valores de ângulo, se aplicável)
def normalize_labels(label):
    max_angle = 25  
    return label / max_angle

# Dataset de treino com transformações atualizadas
train_dataset = CarSimDataset(
    # Se precisar, edite o caminho do driving aqui
    # annotations_file='C:\\Users\\moura\\Searches\\behavior-cloning\\driving_log_ok.csv',
    annotations_file='C:\\Users\\moura\\Searches\\behavior-cloning\\tesst.csv',
    img_dir='DataAndLoader\\IMG',
    transform=transform,
    target_transform=normalize_labels 
)

# Dataset de teste
test_dataset = CarSimDataset(
    annotations_file='C:\\Users\\moura\\Searches\\behavior-cloning\\tesst.csv',
    img_dir='DataAndLoader\\IMG',
    transform=transform,
    target_transform=normalize_labels
)


In [19]:
train_dataset

O código abaixo será utilizado para treinar o modelo com o seu dataset.

In [20]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [21]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 3, 28, 28])
Shape of y: torch.Size([64]) torch.float32


In [22]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28*3, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits.squeeze()

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=2352, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): ReLU()
    (8): Linear(in_features=512, out_features=512, bias=True)
    (9): ReLU()
    (10): Linear(in_features=512, out_features=512, bias=True)
    (11): ReLU()
    (12): Linear(in_features=512, out_features=512, bias=True)
    (13): ReLU()
    (14): Linear(in_features=512, out_features=512, bias=True)
    (15): ReLU()
    (16): Linear(in_features=512, out_features=512, bias=True)
    (17): ReLU()
    (18): Linear(in_features=512, out_features=1, bias=True)
  )
)


In [23]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [24]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [25]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            # correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    # correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [26]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.000053  [   64/22265]
loss: 0.000082  [ 6464/22265]
loss: 0.000025  [12864/22265]
loss: 0.000014  [19264/22265]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.000025 

Epoch 2
-------------------------------
loss: 0.000004  [   64/22265]
loss: 0.000014  [ 6464/22265]
loss: 0.000005  [12864/22265]
loss: 0.000001  [19264/22265]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.000011 

Epoch 3
-------------------------------
loss: 0.000010  [   64/22265]
loss: 0.000003  [ 6464/22265]
loss: 0.000005  [12864/22265]
loss: 0.000001  [19264/22265]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.000009 

Epoch 4
-------------------------------
loss: 0.000016  [   64/22265]
loss: 0.000001  [ 6464/22265]
loss: 0.000007  [12864/22265]
loss: 0.000002  [19264/22265]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.000009 

Epoch 5
-------------------------------
loss: 0.000019  [   64/22265]
loss: 0.000001  [ 6464/22265]
loss: 0.000007  [12864/22265]
loss: 0.000002  [19264/222

In [27]:
torch.save(model.state_dict(), "model_redegigante.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Faça o download do seu modelo após o treinamento, caso queira testá-lo no simulador.

O código a seguir demonstra como o modelo será usado para inferência no simulador. Caso seja necessário, altere a função *preprocess*.

In [28]:
def preprocess(x):
    # TODO: se necessário, alterar função
    return x

In [29]:
# model = NeuralNetwork().to(device)
# model.load_state_dict(torch.load("model.pth"))

In [30]:
# model.eval()
# x = test_data[0][0]
# with torch.no_grad():
    # x = preprocess(x)
    # x = x.to(device)
    # pred = model(x)